In [1]:
import numpy as np
import pandas as pd

from generate_data import load_data
import lifelines
from lifelines import CoxPHFitter
from pycox.datasets import metabric, support

data = 'support'
df = support.read_df()

time = [min(df['duration'])+i*0.01 for i in range(int((max(df['duration'])-min(df['duration']))/0.01))]

train_frac = 0.8
alpha = 0.95
epochs = 100
alphas = [0.6,0.7,0.8,0.9,0.95]
for alpha in alphas:
    coverage = []
    interval_len = []
    for epoch in range(epochs):
          rng = np.random.RandomState(epoch)
          shuffle_idx = rng.permutation(range(len(df)))
          train_idx = shuffle_idx[:int(train_frac*len(df))]
          test_idx = shuffle_idx[int(train_frac*len(df)):]
          df_train = df.iloc[train_idx,:]
          df_test = df.iloc[test_idx,:]
    #       df_train = df_train.drop(columns=['duration_true'])
    #       duration_true = df_test['duration_true']
    #       df_test = df_test.drop(columns=['duration_true'])
          cph = CoxPHFitter()
          cph.fit(df_train,duration_col = 'duration',event_col = 'event')

          surv = cph.predict_survival_function(df_test.iloc[:,:14],times=time)
          surv_ = (surv<=1-alpha).to_numpy(dtype='int8')
          index = np.array(surv.index)
          multiply_surv = np.transpose(surv_)*index
          multiply_surv_ = np.where(multiply_surv==0,np.max(index),multiply_surv)

          t_predict = multiply_surv_.min(axis = 1)
          diff_predict_true = np.subtract(t_predict,np.array(df_test['duration']))

          cover = sum(diff_predict_true>=0)/len(t_predict)

          coverage.append(cover)
          print('[%d]\t%.3f'%(epoch,cover))
          interval_len.append(np.mean(t_predict))
    print('Total Coverage Statistics:\t [Mean]%.3f\t[Std.]%.3f\t[Max]%.3f\t[Min]%.3f'%(np.mean(coverage),np.std(coverage),np.max(coverage),np.min(coverage)))

    np.savetxt('./output/cox_reg_coverage_'+data+'_'+str(epochs)+str(alpha)+'.txt',np.array(coverage))
    np.savetxt('./output/cox_reg_interval_'+data+'_'+str(epochs)+str(alpha)+'.txt',np.array(interval_len))

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


[0]	0.709
[1]	0.694
[2]	0.696
[3]	0.691
[4]	0.708
[5]	0.677
[6]	0.699
[7]	0.705
[8]	0.691
[9]	0.684
[10]	0.708
[11]	0.703
[12]	0.682
[13]	0.687
[14]	0.712
[15]	0.691
[16]	0.696
[17]	0.697
[18]	0.702
[19]	0.705
[20]	0.696
[21]	0.702
[22]	0.660
[23]	0.704
[24]	0.668
[25]	0.687
[26]	0.695
[27]	0.675
[28]	0.693
[29]	0.719
[30]	0.684
[31]	0.695
[32]	0.704
[33]	0.686
[34]	0.686
[35]	0.692
[36]	0.695
[37]	0.697
[38]	0.705
[39]	0.708
[40]	0.714
[41]	0.685
[42]	0.672
[43]	0.695
[44]	0.680
[45]	0.703
[46]	0.683
[47]	0.675
[48]	0.688
[49]	0.688
[50]	0.714
[51]	0.695
[52]	0.696
[53]	0.711
[54]	0.677
[55]	0.674
[56]	0.688
[57]	0.678
[58]	0.701
[59]	0.700
[60]	0.715
[61]	0.700
[62]	0.700
[63]	0.690
[64]	0.718
[65]	0.708
[66]	0.710
[67]	0.703
[68]	0.705
[69]	0.683
[70]	0.706
[71]	0.702
[72]	0.690
[73]	0.691
[74]	0.685
[75]	0.710
[76]	0.692
[77]	0.687
[78]	0.687
[79]	0.691
[80]	0.694
[81]	0.683
[82]	0.706
[83]	0.712
[84]	0.696
[85]	0.701
[86]	0.713
[87]	0.696
[88]	0.708
[89]	0.731
[90]	0.679
[91]	0.69